In [ ]:
using Revise
using Pkg; Pkg.activate("..")

In [ ]:
using Cairn
using LinearAlgebra, Random, Statistics, StatsBase # Do need StatsBase this time
using PotentialLearning
using Molly, AtomsCalculators
using AtomisticQoIs
using SpecialPolynomials

Reference Potential and Domain

In [ ]:
ref = MullerBrownRot()
limits = [[-4.5,1.5],[-2.0,2.0]]

Stuff for Plotting

In [ ]:
using CairoMakie

coord_grid = coord_grid_2d(limits,0.05)
ctr_lvls = -150:50:1000
include("../src/makie/makie.jl")

Obtaining Training Data from MD run 

In [ ]:
sim_langevin = OverdampedLangevin(
                dt =0.002u"ps",
                temperature = 200.0u"K",
                friction=4.0u"ps^-1"
)
start_sys = System(ref,[0.5,0.5], loggers=(coords=CoordinateLogger(100;dims=2),
                                           pe=PotentialEnergyLogger(100),
                                           forces=ForceLogger(100;dims=2)))

In [ ]:
ref_sys = deepcopy(start_sys)
simulate!(ref_sys, sim_langevin, 1_000_000)

initial_num_train = 5000
id = StatsBase.sample(1:length(ref_sys.loggers.coords.history), initial_num_train, replace=false)
train_coords = [ref_sys.loggers.coords.history[i][1] for i in id]
md_trainset = Ensemble(ref,train_coords)

In [ ]:
function plot_pe(system,simulator)
    pe_logger_freq = system.loggers.pe.n_steps
    pe_timestep_freq = ustrip(pe_logger_freq*simulator.dt*1000) # convert to fs

    #I think timesteps start with 0, but I should probably actuallly check
    pe_data = ustrip.(system.loggers.pe.history[:])
    timesteps = 0:pe_timestep_freq:(length(pe_data)-1)*pe_timestep_freq

    pe_fig = Figure(size=(1200,400))
    pe_ax = Axis(pe_fig[1,1],
                 xlabel="Timestep (fs)",
                 ylabel="PE (kJ*mol^-1)")

    scatterlines!(pe_ax,timesteps,pe_data,
                  linewidth=0.1,
                  markersize=3.25)
    pe_fig, pe_data
end

pe_f, pe_data = plot_pe(ref_sys,sim_langevin)
pe_f

In [ ]:
println("Maximum energy: $(maximum(pe_data))\n\
         Minimum energy: $(minimum(pe_data))\n\
         Mean energy   : $(mean(pe_data))")

In [ ]:
function plot_max_abs_forces(system,simulator)
    force_logger_freq = system.loggers.forces.n_steps
    force_timestep_freq = ustrip(force_logger_freq*simulator.dt*1000)
    max_force_data = [maximum(abs.(atom_forces[1])) for atom_forces in system.loggers.forces.history]
    timesteps =0:force_timestep_freq:(length(max_force_data)-1)*force_timestep_freq

    forces_fig = Figure(size=(1200,400))
    forces_ax = Axis(forces_fig[1,1],
                     xlabel="Timestep (fs)",
                     ylabel="Maximum force components")

    scatterlines!(forces_ax,timesteps,max_force_data,
                  linewidth=0.1,
                  markersize=3.25)

    forces_fig, max_force_data
end

forces_f, max_force_data = plot_max_abs_forces(ref_sys,sim_langevin)
forces_f

In [ ]:
println("Maximum force component           : $(maximum(max_force_data))\n\
        Min (per-step max) force component : $(minimum(max_force_data))\n\
        Mean (per-step max) force component: $(mean(max_force_data))")

In [ ]:
f, ax = plot_contours_2d(ref, coord_grid; fill=true, lvls=ctr_lvls)
coordmat = reduce(hcat, [get_values(crd) for crd in train_coords])'
scatter!(ax, coordmat[:,1], coordmat[:,2], color=:red, markersize=5, label="MD train set")
axislegend(ax)
f

Set up and train driving PCE that you are trying to improve

In [ ]:
basisfam = Jacobi{0.5,0.5}
order=20
pce0 = PolynomialChaos(order,2,basisfam,xscl=limits)

In [ ]:
default_ilp = InefficientLearningProblem(;ref=ref)
initial_pce = learn(default_ilp,pce0,md_trainset)

Set up proxy committee potential with lower quality PCE's as well as the committee QoIs

In [ ]:
order_lq = 10
pce_lq = PolynomialChaos(order_lq,2,basisfam,xscl=limits)
cmte_pot = CommitteePotential([let
                                  pce = deepcopy(pce_lq)
                               end for _ in 1:10],1;
                              energy_units = u"kJ*mol^-1",
                              length_units = u"nm")

clp = SubsampleAppendCmteRetrain(InefficientLearningProblem(;ref=ref),
                                 [[-1] for _ in 1:10])

cmte_pot = learn!(clp, cmte_pot,md_trainset;frac=0.7)

In [ ]:
energy_std = CmteEnergy(Statistics.std; strip_units=true)

mean_std_fcomp = CmteFlatForces((cmte=Statistics.std,
                                 coord_and_atom=Statistics.mean); strip_units=true)

In [ ]:
function append_loggers(system::Molly.System, new_loggers::NamedTuple)
    old_keys = keys(system.loggers)
    old_values = values(system.loggers)

    new_keys = keys(new_loggers)
    new_values = values(new_loggers)

    full_keys = (old_keys...,new_keys...)
    full_values = (old_values...,new_values...)

    return NamedTuple{full_keys}(full_values)
end

Set up trial MD to get sense of committee potential thresholds, repeated the MD many times since it quickly becomes unstable, tried to get a gist of where the cmte std for energy and force components spike

In [ ]:
begin
local energy_std_computer(sys,neighbors; kwargs...) = compute(energy_std,sys,cmte_pot)
energy_std_logger = GeneralObservableLogger(energy_std_computer,
                                            Float64,
                                            1)

local mean_std_fcomp_computer(sys,neighbors; kwargs...) = compute(mean_std_fcomp,sys,cmte_pot)
mean_std_fcomp_logger = GeneralObservableLogger(mean_std_fcomp_computer,
                                                Float64,
                                                1)
trial_sys = System(initial_pce,[0.5,0.5];loggers=(coords=CoordinateLogger(1;dims=2),
                                                  pe=PotentialEnergyLogger(1),
                                                  forces=ForceLogger(1;dims=2),
                                                  energy_std=energy_std_logger,
                                                  mean_std_fcomp=mean_std_fcomp_logger,
                                                  ))
end

In [ ]:
simulate!(trial_sys,sim_langevin,50)

In [ ]:
f, ax = plot_contours_2d(ref, coord_grid; fill=true, lvls=ctr_lvls)
trial_coords = [coords[1] for coords in trial_sys.loggers.coords.history]
coordmat = reduce(hcat, [get_values(crd) for crd in trial_coords])'
scatter!(ax, coordmat[:,1], coordmat[:,2], color=:red, markersize=5, label="MD train set")
axislegend(ax)
f

In [ ]:
begin
    system = trial_sys
    simulator = sim_langevin
    pe_logger_freq = system.loggers.pe.n_steps
    pe_timestep_freq = ustrip(pe_logger_freq*simulator.dt*1000) # convert to fs

    #pe_data = ustrip.(abs.(system.loggers.pe.history[1:8]))
    #pe_data = ustrip.(system.loggers.pe.history[1:19])
    pe_data = ustrip.(system.loggers.pe.history)
    timesteps = 0:pe_timestep_freq:(length(pe_data)-1)*pe_timestep_freq

    pe_fig = Figure(size=(1200,400))
    pe_ax = Axis(pe_fig[1,1],
                 xlabel="Timestep (fs)",
                 ylabel="PE (kJ*mol^-1)",
                 limits=(nothing,(-115.0,-102.0))
                 #yscale=log10)
                 )

    scatterlines!(pe_ax,timesteps,pe_data,
                  linewidth=1,
                  markersize=10)
    pe_fig
end

In [ ]:
logger_freq = trial_sys.loggers.energy_std.n_steps
timestep_freq = ustrip(logger_freq*sim_langevin.dt*1000)

#cmte_data = system.loggers.energy_std.history[1:19]
cmte_data = system.loggers.energy_std.history
timesteps = 0:timestep_freq:(length(cmte_data)-1)*timestep_freq

cmte_qoi_fig = Figure(size=(1200,400))
cmte_ax = Axis(cmte_qoi_fig[1,1],
                xlabel="Timestep (fs)",
                ylabel="Energy Std(kJ*mol^-1)",
                limits=(nothing,(0.0,4.0)))

scatterlines!(cmte_ax,timesteps,cmte_data,
              linewidth=1,
              markersize=10)
cmte_qoi_fig

In [ ]:
logger_freq = trial_sys.loggers.mean_std_fcomp.n_steps
timestep_freq = ustrip(logger_freq*sim_langevin.dt*1000)

#cmte_data = system.loggers.mean_std_fcomp.history[1:17]
cmte_data = system.loggers.mean_std_fcomp.history
timesteps = 0:timestep_freq:(length(cmte_data)-1)*timestep_freq

cmte_qoi_fig = Figure(size=(1200,400))
cmte_ax = Axis(cmte_qoi_fig[1,1],
                xlabel="Timestep (fs)",
                ylabel="Energy Std(kJ*mol^-1)",
                limits=(nothing,(0.0,400.0)))

scatterlines!(cmte_ax,timesteps,cmte_data,
              linewidth=1,
              markersize=10)
cmte_qoi_fig

In [ ]:
subtrigger1 = CmteTrigger(energy_std, >, 1.0;
                          logger_spec=(:cmte_energy_std,1))
subtrigger2 = CmteTrigger(mean_std_fcomp,>,100.0;
                          logger_spec=(:cmte_mean_std_fcomp,1))
sct = SharedCmteTrigger((subtrigger1,subtrigger2),deepcopy(cmte_pot))

In [ ]:
coords_eval = potential_grid_2d(ref,[[-4.4,1.5],[-2,2]],0.04,cutoff=800)
sys_eval = Ensemble(ref,coords_eval)
ζ = [ustrip.(Vector(coords)) for coords in coords_eval]
GQint = GaussQuadrature(ζ,ones(length(ζ)) ./length(ζ))
error_spec = Simple2DPotErrors(GQint,true)
dataspec = DefaultALDataSpec(error_spec,true,true,true,true)

In [ ]:
main_mlip = deepcopy(initial_pce)
full_alroutine = ALRoutine(;ref=ref,
                          mlip=main_mlip,
                          trainset=deepcopy(md_trainset),
                          triggers=(sct,),
                          ss=GreedySelector(),
                          lp=InefficientLearningProblem(;ref=ref),
                          trigger_updates=(deepcopy(clp),),
                          sim_update=nothing,
                          aldata_spec=dataspec)

In [ ]:
# Nothing enforces that alroutine.mlip == System.general_inters[1], though that should be the case
main_sys = System(main_mlip, [0.5,0.5];
                  loggers=(coords=CoordinateLogger(1;dims=2),
                           pe=PotentialEnergyLogger(1),
                           forces=ForceLogger(1;dims=2))
                  )

main_sim = OverdampedLangevin(dt =0.001u"ps",
                              temperature = 200.0u"K",
                              friction=4.0u"ps^-1"
                             )

In [ ]:
aldata, post_sys = active_learn!(main_sys,main_sim,100, full_alroutine)

It would be difficult to do multiple active_learn!() routines in a for loop, I think because of the initialization. Maybe pass a flag if system has already been initialized